<a href="https://colab.research.google.com/github/sush19mita/FAKE_REAL_NEWS_PREDICTION/blob/main/real_fake_news_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files
Data = files.upload()

Saving news.csv to news.csv


Importing the Dependencies

In [ ]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

DATA PREPROCESSING

In [ ]:
#loading the dataset to a pandas dataframe
news_dataset = pd.read_csv('/content/news.csv')

In [ ]:
news_dataset.shape

(6335, 4)

In [ ]:
#print the first five rows from the dataset
news_dataset.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
#counting the number of missing values in the dataset
news_dataset.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,0


In [ ]:
#merging the title and text
news_dataset['content'] = news_dataset['title'] + ' ' + news_dataset['text']

In [ ]:
news_dataset['content']

,content
0,You Can Smell Hillary’s Fear Daniel Greenfield...
1,Watch The Exact Moment Paul Ryan Committed Pol...
2,Kerry to go to Paris in gesture of sympathy U....
3,Bernie supporters on Twitter erupt in anger ag...
4,The Battle of New York: Why This Primary Matte...
...,...
6330,State Department says it can't find emails fro...
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332,Anti-Trump Protesters Are Tools of the Oligarc...
6333,"In Ethiopia, Obama seeks progress on peace, se..."


In [ ]:
#separating the data and label
X = news_dataset.drop(columns = 'label', axis = 1)
Y = news_dataset['label']

In [ ]:
print(X)
print(Y)

      Unnamed: 0                                              title  \
0           8476                       You Can Smell Hillary’s Fear   
1          10294  Watch The Exact Moment Paul Ryan Committed Pol...   
2           3608        Kerry to go to Paris in gesture of sympathy   
3          10142  Bernie supporters on Twitter erupt in anger ag...   
4            875   The Battle of New York: Why This Primary Matters   
...          ...                                                ...   
6330        4490  State Department says it can't find emails fro...   
6331        8062  The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...   
6332        8622  Anti-Trump Protesters Are Tools of the Oligarc...   
6333        4021  In Ethiopia, Obama seeks progress on peace, se...   
6334        4330  Jeb Bush Is Suddenly Attacking Trump. Here's W...   

                                                   text  \
0     Daniel Greenfield, a Shillman Journalism Fello...   
1     Google Pinterest Digg L

Stemming procedure


stemming is the procedure of reducing a word to its Root word
example: actor, actress, acting --> act

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content


In [ ]:
news_dataset['content'] =news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0       smell hillari fear daniel greenfield shillman ...
1       watch exact moment paul ryan commit polit suic...
2       kerri go pari gestur sympathi u secretari stat...
3       berni support twitter erupt anger dnc tri warn...
4       battl new york primari matter primari day new ...
                              ...                        
6330    state depart say find email clinton specialist...
6331    p pb stand plutocrat pentagon p pb stand pluto...
6332    anti trump protest tool oligarchi inform anti ...
6333    ethiopia obama seek progress peac secur east a...
6334    jeb bush suddenli attack trump matter jeb bush...
Name: content, Length: 6335, dtype: object


In [ ]:
#separating the data and labels
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [ ]:
print(X)

['smell hillari fear daniel greenfield shillman journal fellow freedom center new york writer focus radic islam final stretch elect hillari rodham clinton gone war fbi word unpreced thrown around often elect ought retir still unpreced nomine major polit parti go war fbi exactli hillari peopl done coma patient wake watch hour cnn hospit bed would assum fbi director jame comey hillari oppon elect fbi attack everyon obama cnn hillari peopl circul letter attack comey current media hit piec lambast target trump surpris clinton alli start run attack ad fbi fbi leadership warn entir left wing establish form lynch mob continu go hillari fbi credibl attack media democrat preemptiv head result investig clinton foundat hillari clinton covert struggl fbi agent obama doj peopl gone explos public new york time compar comey j edgar hoover bizarr headlin jame comey role recal hoover fbi fairli practic admit front spout nonsens boston globe publish column call comey resign outdon time editori claim sca

In [ ]:
print(Y)

['FAKE' 'FAKE' 'REAL' ... 'FAKE' 'REAL' 'REAL']


In [ ]:
#converting textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 43422)	0.030377490785439623
  (0, 43299)	0.01823544823237207
  (0, 42991)	0.03695008224641025
  (0, 42987)	0.0240047103426603
  (0, 42960)	0.033095587951021126
  (0, 42927)	0.023617159002990093
  (0, 42925)	0.008737566380060865
  (0, 42859)	0.01579441180999399
  (0, 42799)	0.016678000766270875
  (0, 42712)	0.02838676929430383
  (0, 42711)	0.016808976313346893
  (0, 42674)	0.038267002580905846
  (0, 42632)	0.0425815201254337
  (0, 42535)	0.03488450955557637
  (0, 42480)	0.035809250475034055
  (0, 42466)	0.018399497997839874
  (0, 42387)	0.013920814295763086
  (0, 42356)	0.06194951909157974
  (0, 42250)	0.01149781838280218
  (0, 42219)	0.027015533479815255
  (0, 42143)	0.02866891016254173
  (0, 42082)	0.032730285384589844
  (0, 42034)	0.016336949950311784
  (0, 41986)	0.053305929475663996
  (0, 41954)	0.07035532559822699
  :	:
  (6334, 3921)	0.028594602233426542
  (6334, 3920)	0.02642484352841616
  (6334, 3742)	0.035328281056127245
  (6334, 3735)	0.020487351480229746
  (6334, 3457)

In [ ]:
# import pandas as pd

# # Assuming your dataset is already loaded into a DataFrame `df`
# # and the column with the 'REAL' and 'FAKE' labels is named 'label'.

# # Display the first few rows to check the column names
# print(news_dataset.head())

# Convert the 'label' column: 'REAL' -> 1, 'FAKE' -> 0
Y = news_dataset['label'].replace({'REAL': 1, 'FAKE': 0})

# Check the first few rows again to confirm the conversion
print(news_dataset['label'])

# Optionally, save the updated DataFrame to a new file
# df.to_csv('/path/to/save/updated_dataset.csv', index=False)


0       0
1       0
2       1
3       0
4       1
       ..
6330    1
6331    0
6332    0
6333    1
6334    1
Name: label, Length: 6335, dtype: int64


Splitting the dataset in testing and training

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, news_dataset['label'],test_size = 0.2, stratify = news_dataset['label'], random_state = 2 )

Training the model

Logistic Regression Model

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation


Accuracy Score

In [ ]:
#accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction , Y_train)

In [ ]:
print("Accuracy of the training data = ", training_data_accuracy )

Accuracy of the training data =  0.9532359905288083


In [ ]:
#accuracy score on the testing data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction , Y_test)

In [ ]:
print("Accuracy of the testing data = ", test_data_accuracy )

Accuracy of the testing data =  0.9171270718232044


MAKING A PREDICTING SYSTEM

In [ ]:
X_new = X_test[1]

prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 1):
  print("The news is real.")
else:
  print("The news is fake.")


[0]
The news is fake.


In [ ]:
# Check if Y_test is a pandas DataFrame or Series
print(type(Y_test))

# If Y_test is a DataFrame or Series, use .iloc[] to access by position
print(Y_test.iloc[1])


<class 'pandas.core.series.Series'>
0
